# Changer la répartition des lexèmes dans les classes flexionnelles et les genres

La redistribution coupe en chunks de même taille pour toutes les combinaisons disponibles pour la catégorie.

- adjectifs : un niveau CF, répartir au hasard
- verbes :
    - le niveau VI/VT/VD n'est pas touché
    - on redistribue sur un niveau CF
- noms : 
    - un niveau genre
        - si genre=A,B,C,D changer les genres et les classes au hasard
        - sinon changer juste les CF au hasard
- autres : pas de modifications

In [1]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import pandas as pd
import itertools as it
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

class NoAliasDumper(yaml.SafeDumper):
    def ignore_aliases(self, data):
        return True

In [2]:
%store -r numerosKalaba
numeroKalaba="23-K%d"%numerosKalaba[0]
print numeroKalaba

23-K1


In [3]:
home = expanduser("~")
serie=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s/"%numeroKalaba

with open(serie+"Stems.yaml", 'r') as stream:
    stems=yaml.safe_load(stream)


In [4]:
def getLexemes(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element]) is list:
#             formesCle=", ".join()
            formes[element]=stemStructure[element]
        else:
            formes.update(getLexemes(stemStructure[element]))
    return formes

In [5]:
def recursive_items(dictionary):
    result={}
    for key, value in dictionary.items():
        if type(value) is dict:
            result[key]=recursive_items(value)
        else:
            result=[]
    return result
        

In [6]:
for genre in stems["NOM"].keys():
    print genre
    cf=stems["NOM"][genre].keys()
    print cf
    lexGenre=getLexemes(stems["NOM"][genre])
    print lexGenre

F
['dakib', 'fedof', 'vjolet', 'dubop', 'klemensja', 'larun', 'guZot', 'Zipos', 'rurovu', 'geDaba', 'tinuk', 'raSiva', 'Sesit', 'biZeb', 'Zeven', 'suZuf', 'komut', 'Sonul', 'fabeZo', 'ruvap', 'zomaNi', 'jasus', 'kemep', 'maira', 'kaZime', 'kader', 'kugoli', 'Sumad', 'nikol', 'elfaba', 'ZiDof', 'katiSa', 'viruD', 'vijen', 'goZuw', 'padis', 'freZa', 'fifiT', 'fazeS', 'agaTos', 'Takik', 'gukepi']
{'dakib': ['chef', 'chefs', 'cheffe', 'cheffes'], 'fedof': ['feu', 'feux'], 'vjolet': ['Violette', 'Violettes'], 'dubop': [u'infirmi\xe8re', u'infirmi\xe8res'], 'klemensja': ['Clemencia', 'Clemencias'], 'larun': ['coyote', 'coyotes'], 'guZot': ['place', 'places'], 'Zipos': [u'col\xe8re', u'col\xe8res'], 'rurovu': ['autruche', 'autruches'], 'geDaba': ['maison', 'maisons'], 'tinuk': [u'obscurit\xe9', u'obscurit\xe9s'], 'raSiva': ['serpent', 'serpents'], 'Sesit': ['chasseuse', 'chasseuses'], 'biZeb': ['livre', 'livres'], 'Zeven': ['lit', 'lits'], 'suZuf': ['souffrance', 'souffrances'], 'komut': ['fe

In [7]:
def cutList(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))


In [8]:
newStems={}
for cat in stems:
    print cat
    cfCat=recursive_items(stems[cat])
    if type(cfCat) is dict:
        if cat=="ADJ":
            newStems["ADJ"]={}
            adjectifs=list(getLexemes(stems["ADJ"]).items())
            adjectifsCF=stems["ADJ"].keys()
#             print adjectifsCF
            for nCF,cfAdjectifs in enumerate(cutList(adjectifs,len(adjectifsCF))):
#                 print {rad:v for rad,v in cfAdjectifs}
                newStems["ADJ"][adjectifsCF[nCF]]={rad:v for rad,v in cfAdjectifs}
        elif cat=="VER":
            newStems["VER"]={}
            if "VI" in cfCat.keys():
                for trans in cfCat.keys():
                    print trans
                    newStems["VER"][trans]={}
                    verbes=list(getLexemes(stems["VER"][trans]).items())
                    verbesCF=stems["VER"][trans].keys()
                    for nCF,cfVerbes in enumerate(cutList(verbes,len(verbesCF))):
    #                     print {rad:v for rad,v in cfVerbes}
                        newStems["VER"][trans][verbesCF[nCF]]={rad:v for rad,v in cfVerbes}
            else:
                verbes=list(getLexemes(stems["VER"]).items())
                verbesCF=stems["VER"].keys()
                for nCF,cfVerbes in enumerate(cutList(verbes,len(verbesCF))):
#                     print {rad:v for rad,v in cfVerbes}
                    newStems["VER"][verbesCF[nCF]]={rad:v for rad,v in cfVerbes}
                
        elif cat=="NOM":
            nOldStems=stems["NOM"]
            newStems["NOM"]={}
            genres=nOldStems.keys()
            nbGenres=len(genres)
            if "A" in genres and "B" in genres:
                print "Genres : ",genres
                noms=list(getLexemes(nOldStems).items())
                # print noms
                cfNoms=set()
                for genre in genres:
                    for cf in nOldStems[genre].keys():
                        cfNoms.add(cf)
                cfNoms=list(cfNoms)
                nbCF=len(cfNoms)
                nomsChunks=list(cutList(noms,nbCF*nbGenres))
                for nGenre,genre in enumerate(genres):
                    newStems["NOM"][genre]={}
                    for nCF,cf in enumerate(cfNoms):
                        popChunk=nomsChunks.pop()
                        # print genre,cf
                        # print popChunk
                        newStems["NOM"][genre][cf]={rad:formes for rad,formes in popChunk}
                        
                
            elif "N1" in nOldStems[genres[0]]:
                for genre in genres:
                    newStems["NOM"][genre]={}
                    cf=nOldStems[genre].keys()
                    nbCF=len(cf)
                    noms=list(getLexemes(nOldStems[genre]).items())
                    nomsChunks=list(cutList(noms,nbCF))
                    nomsCF={"N"+str(k+1):v for k,v in enumerate(nomsChunks)}
                    for cf in nomsCF:
                        newStems["NOM"][genre][cf]={rad:formes for rad,formes in nomsCF[cf]}
            else:
                newStems["NOM"]=nOldStems
                        
        else:
            newStems[cat]=stems[cat]
    else:
        newStems[cat]=stems[cat]
print newStems

NOM
VER
ADJ
PRO
DET
PREP
{'NOM': {'F': {'dakib': ['chef', 'chefs', 'cheffe', 'cheffes'], 'fedof': ['feu', 'feux'], 'vjolet': ['Violette', 'Violettes'], 'dubop': [u'infirmi\xe8re', u'infirmi\xe8res'], 'klemensja': ['Clemencia', 'Clemencias'], 'larun': ['coyote', 'coyotes'], 'guZot': ['place', 'places'], 'Zipos': [u'col\xe8re', u'col\xe8res'], 'rurovu': ['autruche', 'autruches'], 'geDaba': ['maison', 'maisons'], 'tinuk': [u'obscurit\xe9', u'obscurit\xe9s'], 'raSiva': ['serpent', 'serpents'], 'Sesit': ['chasseuse', 'chasseuses'], 'biZeb': ['livre', 'livres'], 'Zeven': ['lit', 'lits'], 'suZuf': ['souffrance', 'souffrances'], 'komut': ['femme', 'femmes'], 'Sonul': ['lever-N', 'lever', 'levers'], 'fabeZo': ['table', 'tables'], 'ruvap': ['disparition', 'disparitions'], 'zomaNi': ['chatte', 'chattes'], 'jasus': [u'm\xe8re', u'm\xe8res'], 'kemep': [u'c\xf4t\xe9', u'c\xf4t\xe9s'], 'maira': ['Mahira', 'Mahiras'], 'kaZime': [u'lumi\xe8re', u'lumi\xe8res'], 'kader': ['louve', 'louves'], 'kugoli': [

In [9]:
# newStems["NOM"]["B"]

In [10]:
with open(serie+'FlowStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True,Dumper=NoAliasDumper)

In [11]:
with open(serie+"FlowStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [12]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  Donum: [blanc, blancs, blanche, blanches]
  Neval: [cinq]
  NigeS: [rouge, rouges]
  NiruN: [deux]
  Nuzer: [trois]
  ToSaS: [furieux, furieuse, furieuses]
  Tunas: [terrible, terribles]
  Zigon: [gros, grosse, grosses]
  denaZe: [vilain, vilains, vilaine, vilaines]
  gemabe: [différent, différents, différente, différentes]
  gimil: [dernier, derniers, dernière, dernières]
  gubuk: [vert, verts, verte, vertes]
  gumaz: [treize]
  kiDite: [bas, basse, basses]
  laroN: [petit, petits, petite, petites]
  liZan: [suivant, suivants, suivante, suivantes]
  lujef: [quatre]
  patoS: [courageux, courageuse, courageuses]
  poTiko: [nombreux, nombreuses]
  rapiZ: [méchant, méchants, méchante, méchantes]
  remes: [grand, grands, grande, grandes]
  rikim: [jaune, jaunes]
  rivuk: [effrayé, effrayés, effrayée, effrayées]
  rugego: [six]
  ruguk: [disparu, disparus, disparue, disparues]
  rurub: [blessé, blessés, blessée, blessées]
  siziS: [troisième, troisièmes]
  taTuk: [maigre, maigres]
  

In [13]:
with open(serie+'NewStems.yaml', 'w') as output:
    output.write(yamlText)

In [14]:
ding()